In [1]:
import firebase_admin
from firebase_admin import firestore
from firebase_admin import credentials

# Run once
cred = credentials.Certificate('ai-writing-45138-f0d64e9114cc.json')
app = firebase_admin.initialize_app(cred)
db = firestore.client()

In [6]:
import pandas as pd

# def get_all_documents():
df = pd.DataFrame()
docs = db.collection('user-input').stream()
i=0
for doc in docs:
    i+=1
    print(f'{doc.id} => {doc.to_dict()}')
    if i==4:
        break

123 => {'postStudyAnswers': {'set1': [1, 1, 1, 'first negative', '', '', 2, 2, 2, 'second positive', '213124', '123'], 'set2': [1, 1, 1, 1, 1, 2, 2, 2, 2, 2]}, 'option': 2, 'numActions': {'2-neg': {'Generate': 1, 'Ignore': 1, 'Accept': 0, 'Regenerate': 0}, '3-pos': {'Generate': 0, 'Ignore': 0, 'Accept': 0, 'Regenerate': 0}}, 'demographicQuestions': {'q3': '1', 'q6.3': 2, 'q1': 'Some high school, no diploma', 'q4': '1', 'q5': '1', 'q6.2': 2, 'q8': '1', 'q2': '1', 'q6.1': 2, 'q7': 2}, 'aiWritingText': {'2-neg': 'this is the first task - colleague task', '3-pos': '21435345'}, 'logs': {'2-neg': [{'eventTimestamp': 1726164826036, 'textDelta': 't', 'currentCursor': 0, 'eventName': 'text-insert'}, {'eventTimestamp': 1726164826118, 'textDelta': 'h', 'currentCursor': 1, 'eventName': 'text-insert'}, {'eventTimestamp': 1726164826185, 'textDelta': 'i', 'currentCursor': 2, 'eventName': 'text-insert'}, {'eventTimestamp': 1726164826344, 'textDelta': 's', 'currentCursor': 3, 'eventName': 'text-insert'

In [80]:
content = doc.to_dict()

# demographic
def format_demo(dic):
    custom_order = ['q1', 'q1-other','q2', 'q3','q3','q4','q5','q6.1','q6.2','q6.3','q7','q8']
    ordered_dic = {key: dic.get(key) for key in custom_order}
    ordered_dic = {key.replace('q','D'): ordered_dic[key] for key in ordered_dic}
    return ordered_dic

demo_df = format_demo(content['demographicQuestions'])
pd.DataFrame([demo_df])

,D1,D1-other,D2,D3,D4,D5,D6.1,D6.2,D6.3,D7,D8
0,"Some high school, no diploma",None,1,1,1,1,2,2,2,2,1


In [70]:
# pre-study
def format_pre_study(dic):
    custom_order = ['1','2','3','4','5','6','7','8','9','10']
    new_dic = {}
    for key in custom_order:
        lst = dic.get(key)
        new_cols = {}
        for i in range(3):
            new_cols['PRE'+key+'.'+str(i+1)] = lst[i]
        new_dic.update(new_cols)
    return new_dic

pre_study_df = format_pre_study(content['preStudyAnswers'])
pd.DataFrame([pre_study_df])

,PRE1.1,PRE1.2,PRE1.3,PRE2.1,PRE2.2,PRE2.3,PRE3.1,PRE3.2,PRE3.3,PRE4.1,...,PRE7.3,PRE8.1,PRE8.2,PRE8.3,PRE9.1,PRE9.2,PRE9.3,PRE10.1,PRE10.2,PRE10.3
0,5,4,4,4,4,5,5,4,3,7,...,3,7,3,2,3,4,5,5,4,3


In [122]:
def format_post_study(dic, n):
    new_dic = {}
    set1 = dic['set1']
    set2 = dic['set2']
    q_lst = ['S','P','D']
    prefix = 'POST' + str(n+1) + '.'
    for i in range(3):
        key = prefix + q_lst[i]
        idx = i+6*n
        new_dic[key] = set1[idx]
        new_dic[key+'-reason'] = set1[idx+3]
    for i in range(5):
        key = prefix + str(i+1)
        new_dic[key] = set2[n*5+i]
    return new_dic

def format_ai_text(dic):
    new_dic = {}
    texts = dic['aiWritingText']
    post = dic['postStudyAnswers']
    keys = list(texts.keys())
    for i in range(2):
        num = str(i+1)
        key = keys[i]
        new_dic['AI Task '+num] = key
        new_dic['AI Text '+num] = texts.get(key)
        new_dic |= format_post_study(post,i)

    return new_dic

dic = format_ai_text(content)
dic

{'AI Task 1': '2-neg',
 'AI Text 1': 'this is the first task - colleague task',
 'POST1.S': 1,
 'POST1.S-reason': 'first negative',
 'POST1.P': 1,
 'POST1.P-reason': '',
 'POST1.D': 1,
 'POST1.D-reason': '',
 'POST1.1': 1,
 'POST1.2': 1,
 'POST1.3': 1,
 'POST1.4': 1,
 'POST1.5': 1,
 'AI Task 2': '3-pos',
 'AI Text 2': '21435345',
 'POST2.S': 2,
 'POST2.S-reason': 'second positive',
 'POST2.P': 2,
 'POST2.P-reason': '213124',
 'POST2.D': 2,
 'POST2.D-reason': '123',
 'POST2.1': 2,
 'POST2.2': 2,
 'POST2.3': 2,
 'POST2.4': 2,
 'POST2.5': 2}

In [84]:
df = pd.DataFrame()
docs = db.collection('user-input').stream()

for doc in docs:
    content = doc.to_dict()
    new_content = {}
    new_content['ID'] = doc.id
    demo_dic = format_demo(content['demographicQuestions'])
    pre_study_dic = format_pre_study(content['preStudyAnswers'])
    new_content |= demo_dic | pre_study_dic
    new_content['Baseline Text'] = content['baselineText']
    # new_content['AI Text 1'] = content['aiWritingText'][0]
    # new_content['AI Text 2'] = content['aiWritingText'][1]
    # print(f'{doc.id} => {doc.to_dict()}')
    break

pd.DataFrame([new_content])


KeyError: 0

In [2]:
get_all_documents()

123 => {'postStudyAnswers': {'set1': [1, 1, 1, 'first negative', '', '', 2, 2, 2, 'second positive', '213124', '123'], 'set2': [1, 1, 1, 1, 1, 2, 2, 2, 2, 2]}, 'option': 2, 'numActions': {'2-neg': {'Generate': 1, 'Ignore': 1, 'Accept': 0, 'Regenerate': 0}, '3-pos': {'Generate': 0, 'Ignore': 0, 'Accept': 0, 'Regenerate': 0}}, 'demographicQuestions': {'q3': '1', 'q6.3': 2, 'q1': 'Some high school, no diploma', 'q4': '1', 'q5': '1', 'q6.2': 2, 'q8': '1', 'q2': '1', 'q6.1': 2, 'q7': 2}, 'aiWritingText': {'2-neg': 'this is the first task - colleague task', '3-pos': '21435345'}, 'logs': {'2-neg': [{'eventTimestamp': 1726164826036, 'textDelta': 't', 'currentCursor': 0, 'eventName': 'text-insert'}, {'eventTimestamp': 1726164826118, 'textDelta': 'h', 'currentCursor': 1, 'eventName': 'text-insert'}, {'eventTimestamp': 1726164826185, 'textDelta': 'i', 'currentCursor': 2, 'eventName': 'text-insert'}, {'eventTimestamp': 1726164826344, 'textDelta': 's', 'currentCursor': 3, 'eventName': 'text-insert'